<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/process_control_with_neqsim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Process Control in NeqSim
#@markdown This notebook introduces basic process control concepts and shows how they can be implemented in the [NeqSim](https://neqsim.github.io/neqsimpython/) process simulator.
%%capture
!pip install neqsim matplotlib
from neqsim import jneqsim
import matplotlib.pyplot as plt
from IPython.display import YouTubeVideo


In [ ]:
#@title Video: Process Control Basics
#@markdown A short introduction to process control concepts.
YouTubeVideo('FEnwYVPDRDE', width=600, height=400)

In [ ]:
#@title Video: PID Controller Explained
#@markdown Understanding proportional-integral-derivative control.
YouTubeVideo('UR0hOmjaHp0', width=600, height=400)

## Process control theory
Process control keeps process variables such as pressure, flow or level close to desired setpoints. Feedback controllers (P, PI and PID) compare a measured value with the setpoint and manipulate an actuator to reduce the error.


## Control options in NeqSim
NeqSim supports dynamic simulation with measurement devices (transmitters) and controller devices. Controllers can be attached to valves or other equipment and tuned by specifying proportional (K$_p$), integral (T$_i$) and derivative (T$_d$) parameters.


## Example: controlling separator level and pressure
The example below shows a simple separator with control of both liquid level and gas outlet pressure. A level transmitter and a pressure transmitter provide measurements which are used by PID controllers connected to outlet valves.


In [ ]:
# define the feed fluid
fluid1 = jneqsim.thermo.system.SystemSrkEos(298.15, 10.0)
fluid1.addComponent("methane", 0.9)
fluid1.addComponent("ethane", 0.1)
fluid1.addComponent("n-heptane", 1.0)
fluid1.setMixingRule('classic')

# build process equipment
feed = jneqsim.process.equipment.stream.Stream("Feed", fluid1)
feed.setFlowRate(50.0, "kg/hr")
feed.setPressure(10.0, "bara")

inlet_valve = jneqsim.process.equipment.valve.ThrottlingValve("inlet valve", feed)
inlet_valve.setOutletPressure(5.0)
inlet_valve.setPercentValveOpening(30)

sep = jneqsim.process.equipment.separator.Separator("Separator")
sep.addStream(inlet_valve.getOutletStream())
sep.setSeparatorLength(0.3)
sep.setInternalDiameter(1.0)
sep.setLiquidLevel(0.5)

liq_valve = jneqsim.process.equipment.valve.ThrottlingValve("liquid valve", sep.getLiquidOutStream())
liq_valve.setOutletPressure(1.0)
liq_valve.setPercentValveOpening(50)

gas_valve = jneqsim.process.equipment.valve.ThrottlingValve("gas valve", sep.getGasOutStream())
gas_valve.setOutletPressure(1.0)
gas_valve.setPercentValveOpening(50)

# measurement devices
level_trans = jneqsim.process.measurementdevice.LevelTransmitter(sep)
level_trans.setMaximumValue(0.8)
level_trans.setMinimumValue(0.2)

pressure_trans = jneqsim.process.measurementdevice.PressureTransmitter(sep.getGasOutStream())
pressure_trans.setUnit("bar")
pressure_trans.setMaximumValue(10.0)
pressure_trans.setMinimumValue(1.0)

# controllers
level_contr = jneqsim.process.controllerdevice.ControllerDeviceBaseClass()
level_contr.setReverseActing(False)
level_contr.setTransmitter(level_trans)
level_contr.setControllerSetPoint(0.45)
level_contr.setControllerParameters(1.0, 200.0, 0.0)

pressure_contr = jneqsim.process.controllerdevice.ControllerDeviceBaseClass()
pressure_contr.setReverseActing(False)
pressure_contr.setTransmitter(pressure_trans)
pressure_contr.setControllerSetPoint(5.0)
pressure_contr.setControllerParameters(1.0, 50.0, 0.0)

liq_valve.setController(level_contr)
gas_valve.setController(pressure_contr)

# assemble and run process
p = jneqsim.process.processmodel.ProcessSystem()
p.add(feed)
p.add(inlet_valve)
p.add(sep)
p.add(liq_valve)
p.add(gas_valve)
p.add(level_trans)
p.add(pressure_trans)
p.run()

# dynamic simulation
p.setTimeStep(1.0)
time=[]; level=[]; pressure=[]
for i in range(20):
    time.append(p.getTime())
    level.append(level_trans.getMeasuredValue())
    pressure.append(pressure_trans.getMeasuredValue())
    p.runTransient()

pressure_contr.setControllerSetPoint(4.0)
for i in range(20):
    time.append(p.getTime())
    level.append(level_trans.getMeasuredValue())
    pressure.append(pressure_trans.getMeasuredValue())
    p.runTransient()

plt.plot(time, level, label='Separator level')
plt.plot(time, pressure, label='Separator pressure (bar)')
plt.xlabel('Time [s]')
plt.legend()
plt.show()


The plot shows how the PID controllers keep level and pressure close to their setpoints and how the pressure responds when the setpoint is changed from 5 to 4 bar.
